In [1]:
import pandas as pd
import re

In [2]:
f = open('WhatsApp Chat with Subject.txt', 'r', encoding='utf-8')

In [3]:
data = f.read()

In [ ]:
print(data)

pattern = r'\d{1,2}/\d{1,2}/\d{2,4},\s\d{1,2}:\d{2}\s-\s'

In [56]:
messages = re.split(pattern, data)[2:]
# messages
len(messages)

1588

In [57]:
dates = re.findall(pattern, data)[1:]
# dates
len(dates)

1588

In [ ]:
df = pd.DataFrame({'user_message': messages, 'message_date': dates})

# convert messsage_date type
df['message_date'] = pd.to_datetime(df['message_date'], format='%d/%m/%Y, %H:%M - ')

df.rename(columns = {'message_date': 'date'}, inplace = True)
df.head()

In [59]:
df.shape

(1588, 2)

In [ ]:
# separate users and messages
users = []
messages = []
for message in df['user_message']:
    entry = re.split(r'([\w\W]+?):\s', message)
    # print(entry)
    if entry[1:]: # username
        users.append(entry[1])
        messages.append(entry[2])
    else:
        users.append('whatsapp_notification')
        messages.append(entry[0])

# print (len(messages))
df['user'] = users
df['message'] = messages
df.drop(columns=['user_message'], inplace=True)

df.head()